## linkes for the project

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow.svg)](https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3)

[![GitHub Repoistory](https://badges.aleen42.com/src/github.svg)](https://github.com/TariqAlhathloul/Smart-Street)


## installation

In [1]:
!pip install ultralytics -q
!pip install roboflow -q

In [2]:
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd 
import os

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("Roboflow_API_KEY")

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-3")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.98, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Road-Lines-Segmentation-3-1 in yolov8:: 100%|██████████| 34020/34020 [00:04<00:00, 7655.08it/s]


## upload the dataset

In [4]:
TRAIN_DIR_IAMAGES = '/kaggle/working/Road-Lines-Segmentation-3-1/train/images'
VAL_DIR_IAMGES = '/kaggle/working/Road-Lines-Segmentation-3-1/valid/images'
TEST_DIR_IMAGES = '/kaggle/working/Road-Lines-Segmentation-3-1/test/images'

In [5]:
print(f"Total Training: {len(os.listdir(TRAIN_DIR_IAMAGES))} images")
print(f"Total Validation: {len(os.listdir(VAL_DIR_IAMGES))} images")
print(f"Total Testing: {len(os.listdir(TEST_DIR_IMAGES))} images")

Total Training: 14145 images
Total Validation: 1845 images
Total Testing: 1014 images


## check if the GPU is avalabie

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 22 08:32:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## upload the model

In [6]:
#model = YOLO('/kaggle/input/best/pytorch/default/1/best.pt')
# first experiment using the pervious model from the mini project

In [7]:
# second experiment usning yolov8n-eg
model = YOLO("yolov8n-seg.pt")

100%|██████████| 6.74M/6.74M [00:00<00:00, 78.5MB/s]


## overwrite the data.yaml file

In [8]:
%%writefile /kaggle/working/Road-Lines-Segmentation-3-1/data.yaml
names:
- bus
- car
- license_plate
- solid-yellow-line
- truck
nc: 5
roboflow:
  license: CC BY 4.0
  project: road-lines-segmentation-3
  url: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3/dataset/1
  version: 1
  workspace: capstoneteam
test: /kaggle/working/Road-Lines-Segmentation-3-1/test/images
train: /kaggle/working/Road-Lines-Segmentation-3-1/train/images
val: /kaggle/working/Road-Lines-Segmentation-3-1/valid/images

Overwriting /kaggle/working/Road-Lines-Segmentation-3-1/data.yaml


## Hyperparameters, finetuning

In [9]:
IMG_SIZE = 640
BATCH_SIZE = -1
EPOCHS = 100

In [10]:
results = model.train(
    data = '/kaggle/working/Road-Lines-Segmentation-3-1/data.yaml',
    task = 'segment',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Segment-Model-5',
    project = 'experiment-5',
    patience = 10,
    save = True,
    optimizer = 'SGD',
    lr0= 0.01,
    lrf= 0.01,
    dropout=0.2
)

Ultralytics YOLOv8.2.98 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/kaggle/working/Road-Lines-Segmentation-3-1/data.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=experiment-5, name=Segment-Model-5, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=93, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]
2024-09-22 08:33:01,770	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-22 08:33:02,345	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...


33 epochs completed in 2.158 hours.
Optimizer stripped from experiment-5/Segment-Model-5/weights/last.pt, 6.8MB
Optimizer stripped from experiment-5/Segment-Model-5/weights/best.pt, 6.8MB

Validating experiment-5/Segment-Model-5/weights/best.pt...
Ultralytics YOLOv8.2.98 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,259,039 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:21<00:00,  1.37it/s]


                   all       1845       7143      0.727      0.621      0.672      0.546      0.692      0.586      0.631      0.461
                   bus        199        200      0.827       0.67      0.796      0.712      0.839      0.665      0.797      0.686
                   car       1676       4680      0.855      0.844      0.916      0.772      0.856      0.837      0.909      0.659
         license_plate        188        277       0.54      0.404      0.377      0.161      0.341      0.242      0.181     0.0525
     solid-yellow-line       1768       1923      0.854      0.927      0.942       0.82      0.864      0.931      0.942      0.647
                 truck         52         63      0.559      0.261      0.328      0.264      0.558      0.254      0.327      0.263
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to experiment-5/Segment-Model-5


lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▁▅█████▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅
lr/pg2,▁▅█████▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅
metrics/mAP50(B),▃▅▁▄▅▄▅▆▅▆▇▆▇▆▇▆▆▆▇▇▆▆█▆▆▇▆▇▆▆▆▆█
metrics/mAP50(M),▄▅▁▄▆▅▆▆▆▇▇▆▇▆█▆▆▆▇█▆▇█▇▇▇▆▇▆▇▇▇█
metrics/mAP50-95(B),▃▅▁▃▅▄▅▆▅▆▇▆▆▆▇▆▆▆▇▇▆▇█▇▇▇▇▇▆▇▇▇█
metrics/mAP50-95(M),▄▅▁▄▅▄▅▆▆▇▇▅▆▆▇▆▆▇▇▇▆▇█▇▇▇▇█▇▇▇▇█
metrics/precision(B),█▃▁▅▅▃▄▅▅▆▇▅▄▄▅▆▄▅▅▅▄▅▇█▅▆▅▆▅▆▆▅▇
metrics/precision(M),█▂▁▄▅▃▃▅▄▅▆▃▃▃▅▅▃▄▅▄▃▄▅▆▄▅▄▄▄▅▅▆▅
metrics/recall(B),▁▆▁▃▄▄▆▇▅▅▅▅█▇▆▄▆▂▇▇▆▄▇▃▅▆▄▅▂▅▃▄▇
metrics/recall(M),▁▇▁▄▅▄▇▇▆▅▅▅█▇▆▄▆▃▆▇▅▅▆▄▅▆▅▆▃▅▄▃▆


os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


## evaluation on the validationset


In [11]:
evaluation = model.val(
    data = '/kaggle/working/Road-Lines-Segmentation-3-1/data.yaml',
    task = 'segment',
    imgsz = 640,
    iou = 0.6,
    batch = 32,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-evaluation-5',
    save_json = True
)

Ultralytics YOLOv8.2.98 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,259,039 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning /kaggle/working/Road-Lines-Segmentation-3-1/valid/labels.cache... 1845 images, 5 backgrounds, 0 corrupt: 100%|██████████| 1845/1845 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/Road-Lines-Segmentation-3-1/valid/images/front-left-NO20240903-145041-000844F_MP4-0244_jpg.rf.d35bf3f6a9afb53703cdd639f4526b98.jpg: 1 duplicate labels removed



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/58 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['pycocotools>=2.0.6'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 7.6 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 12.8s, installed 1 package: ['pycocotools>=2.0.6']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [01:11<00:00,  1.23s/it]


                   all       1845       7143      0.734      0.625      0.677      0.549      0.712      0.609      0.654      0.449
                   bus        199        200      0.828      0.675      0.796      0.711      0.827      0.675      0.799      0.682
                   car       1676       4680      0.854      0.845      0.918      0.777      0.853      0.845      0.917      0.678
         license_plate        188        277      0.566      0.415      0.403      0.165       0.45      0.332      0.286     0.0791
     solid-yellow-line       1768       1923      0.863      0.926       0.94      0.822      0.868      0.931      0.942      0.544
                 truck         52         63      0.559      0.262      0.329      0.268      0.561      0.264      0.328       0.26
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving experiment-5/Model-segment-evaluation-5/predictions.json...
Results saved to experiment-5/Model-segment-evaluatio

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


## predictions on the testset

In [12]:
predictions = model.predict(
    data = '/kaggle/working/Road-Lines-Segmentation-3-1/data.yaml',
    source = '/kaggle/working/Road-Lines-Segmentation-3-1/test/images',
    task = 'segment',
    imgsz = 640,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-predictions',
    save_json = True,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1014 /kaggle/working/Road-Lines-Segmentation-3-1/test/images/NO20240828-152333-000152F_MP4-0008_jpg.rf.887ba244a440edadb93ef7968520f916.jpg: 640x640 6 cars, 9.0ms
image 2/1014 /kaggle/working/Road-Lines-Segmentation-3-1/test/images/NO20240828-152333-000152F_MP4-0075_jpg.rf.7773aeaf1934897b09a365966051ed5b.jpg: 640x640 6 cars, 1 license_plate, 6.9ms
image 3/1014 /kaggle/working/Road-Lines-Segmentation-3-1/test/images/NO20240828-152333-000152F_MP

## apply quantization

In [14]:
model.export(format="onnx", int8=True, dynamic=True,half=True, simplify=True, data='/kaggle/working/Road-Lines-Segmentation-3-1/data.yaml')

Ultralytics YOLOv8.2.98 🚀 Python-3.10.14 torch-2.4.0 CPU (Intel Xeon 2.00GHz)
WARNING ⚠️ half=True and int8=True are mutually exclusive, setting half=False.

PyTorch: starting from 'experiment-5/Segment-Model-5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 41, 8400), (1, 32, 160, 160)) (6.5 MB)
requirements: Ultralytics requirements ['onnxslim==0.1.34', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 219.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 187.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 215.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 19.9s, installed 2 packages: ['onnxslim==0.1.34', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.2 

'experiment-5/Segment-Model-5/weights/best.onnx'

In [15]:
import shutil

shutil.make_archive('/kaggle/working/experiment-5', 'zip', '/kaggle/working/')

'/kaggle/working/experiment-5.zip'

In [17]:
!pip install ultralytics==8.0.196 -q

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


In [18]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-3")
project.version(1).deploy(model_type="yolov8n", model_path="/kaggle/working/experiment-5/Segment-Model-5")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.98, to fix: `pip install ultralytics==8.0.196`


Would you like to continue with the wrong version of ultralytics? y/n:  y


View the status of your deployment at: https://app.roboflow.com/capstoneteam/road-lines-segmentation-3/1
Share your model with the world at: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3/model/1
